In [1]:
import http.client, urllib

PUSHOVER_USER='uf1zq9jv4o73nrk9m4ccbrb56h3q5i'
PUSHOVER_TOKEN='a28oao5kfvsatehcpi1jbgoqiskt56'

def test_push():
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
        urllib.parse.urlencode({
            "token": PUSHOVER_TOKEN,
            "user": PUSHOVER_USER,
            "message": "🚀 This is a test notification from Python!",
            "sound": "cashregister"
        }), { "Content-type": "application/x-www-form-urlencoded" })
    print(conn.getresponse().read())

test_push()


b'{"status":1,"request":"8e54fa9a-c47c-4013-8b2c-d469a77710b0"}'


In [3]:
# Cell: Test Ollama and assemble NDJSON chunks into one message
import requests
import json

ollama_url   = "http://127.0.0.1:11436/api/chat"
ollama_model = "llama3.2"
payload = {
    "model": ollama_model,
    "messages": [
        {"role": "system",  "content": "You are a test."},
        {"role": "user",    "content": "Ping"}
    ]
}

try:
    # Use stream=True to iterate line by line
    resp = requests.post(ollama_url, json=payload, stream=True, timeout=5)
    print("Status Code:", resp.status_code)
    
    assembled = ""
    print("\n--- NDJSON CHUNKS ---")
    for line in resp.iter_lines(decode_unicode=True):
        if not line:
            continue
        print(line)  # show each raw JSON line
        try:
            msg = json.loads(line)
            # accumulate the assistant's 'content' field
            assembled += msg.get("message", {}).get("content", "")
        except json.JSONDecodeError as je:
            print("  ⚠️ JSON decode error:", je)
    
    print("\n--- ASSEMBLED CONTENT ---")
    print(assembled)
    
except Exception as e:
    print("Request failed:", e)


Status Code: 200

--- NDJSON CHUNKS ---
b'{"model":"llama3.2","created_at":"2025-07-27T16:54:05.9282356Z","message":{"role":"assistant","content":"pong"},"done":false}'
b'{"model":"llama3.2","created_at":"2025-07-27T16:54:05.9481977Z","message":{"role":"assistant","content":""},"done_reason":"stop","done":true,"total_duration":342759300,"load_duration":122444000,"prompt_eval_count":31,"prompt_eval_duration":30257000,"eval_count":2,"eval_duration":184769400}'

--- ASSEMBLED CONTENT ---
pong
